# Initial AI Agent Impala w CNN (working) MR2 HS Disc

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 4.0
#my_config["ep_max_length"] = 200
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = False
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 1,
  'agent' : 'APPO',
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for APPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 1 is High Speed Ring, 2 is 0-400m, 
}

In [ ]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [ ]:
import ray
ray.shutdown()
ray.init()

In [ ]:
from ray.rllib.algorithms.impala.impala import ImpalaConfig
config = (
    ImpalaConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        #train_batch_size=128,
        )
)

#APPOConfig.framework("")

In [ ]:
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        try:
            json.dumps(obj)
        except TypeError:
            return "Not serializable object!"

        return obj

config_dict = config.to_dict()

print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [ ]:
algo = config.build()

In [ ]:
algo.get_policy().model

In [9]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


KeyboardInterrupt: 